In [2]:
import MDAnalysis as mda

In [21]:
from numpy import exp, log, arange, array, float32, unique, sqrt, where, ndarray, isin, dtype, concatenate
from copy import deepcopy

def compute_se(e1:float,e2:float,s1:float,s2:float):
   eij = (float32(e1)*float32(e2))**0.5
   sij = 0.5*(float32(s1)+float32(s2))
   return eij,sij

def generate_nonbonded(at1, at2):
   e1 = float32(at1[-1])
   e2 = float32(at2[-1])
   s1 = float32(at1[-2])
   s2 = float32(at2[-2])
   funct = str(1)
   eij, sij = compute_se(e1,e2,s1,s2)
   a1, a2 = at1[0], at2[0]
   return f'{a1:>5} {a2:>4} {funct:^5} {sij:>10.4f} {eij:>8.4f}\n'

def query_typed(query, qtype):
   return eval(qtype)(input(query))

def molecule_selections(queryN:list, queryM:list):
   response = query_typed(*queryN)
   molecule_sel = []
   for i in range(response):
      molecule_sel.append(query_typed(*queryM))
   return molecule_sel

def _parse_section(readfile,trunks):
   first_round = True
   output = []
   for line in readfile[trunks:]:
      if "[" not in line and first_round!=True and len(line.split()) != 0:
         output.append(line)
      elif ";" == line[0]: 
         continue
      elif "[" in line and first_round!=True: 
         break
      first_round = False
   return output

def get_molecule_atomtypes(sections,molecule:int=0):
   atomtype_list = []
   for i in sections['moleculetype'][molecule]['atoms']:
      if len(i.split())>1 and ';' not in i.split()[0] and '[' not in i :
         atomtype_list.append(i.split()[:2])
   dataset = array(atomtype_list,dtype=object)
   atomtypes = dataset[:,1]
   return unique(atomtypes)

def dihedraltype_array(dihedraltypes:list):
   data = []
   dt = dtype([('i','U10'),
                  ('j', 'U10'),  # 2nd string
                  ('k', 'U10'),  # 3rd string
                  ('l', 'U10'),  # 4th string
                  ('func', 'i4'),    # 5th integer
                  ('angle', 'f4'),    # 6th float
                  ('K', 'f4'),    # 7th float
                  ('mult', 'i4')])   # 8th integer])
   for string in dihedraltypes:
      line = string[:string.find(';')].split()
      data.append((line[0], line[1], line[2], line[3], int(line[4]), float(line[5]), float(line[6]), int(line[7])))
   return array(data, dtype=dt)

def dihedral_array(dihedraltypes:ndarray, dihedrals:list, atom_mapping:dict, verbose=False):
   data = []
   search_dihedrals = []
   dtype_dihedrals = dtype([('i','i4'),
                  ('j', 'i4'),  # 2nd string
                  ('k', 'i4'),  # 3rd string
                  ('l', 'i4'),  # 4th string
                  ('func', 'i4'),    # 5th integer
                  ('angle', 'f4'),    # 6th float
                  ('K', 'f4'),    # 7th float
                  ('mult', 'i4')])   # 8th integer])
   for string in dihedrals:
      line = string[:string.find(';')].split()
      if len(line) == 5:
         search_dihedrals.append(tuple(map(int,line)))
      elif len(line) == 8:
         data.append((int(line[0]), int(line[1]), int(line[2]), int(line[3]), int(line[4]), float(line[5]), float(line[6]), int(line[7])))
      elif not line:
         continue
      else:
         raise RuntimeError(f'Line not processed: {line}')
   for (i, j, k, l, func) in search_dihedrals:
      if func == 4:
         mask = ((isin(dihedraltypes['i'], atom_mapping[i]) & \
               isin(dihedraltypes['j'], atom_mapping[j]) & \
               isin(dihedraltypes['k'], atom_mapping[k]) & \
               isin(dihedraltypes['l'], atom_mapping[l]) & \
               isin(dihedraltypes['func'], func)) | \
               (isin(dihedraltypes['i'], atom_mapping[l]) & \
               isin(dihedraltypes['j'], atom_mapping[k]) & \
               isin(dihedraltypes['k'], atom_mapping[j]) & \
               isin(dihedraltypes['l'], atom_mapping[i]) & \
               isin(dihedraltypes['func'], func))) & isin(dihedraltypes['func'], func)
         mask_Xl = (isin(dihedraltypes['i'], 'X') & \
               isin(dihedraltypes['j'], atom_mapping[j]) & \
               isin(dihedraltypes['k'], atom_mapping[k]) & \
               isin(dihedraltypes['l'], atom_mapping[l]) & \
               isin(dihedraltypes['func'], func)) 
         mask_Xll = (isin(dihedraltypes['i'], 'X') & \
               isin(dihedraltypes['j'], 'X') & \
               isin(dihedraltypes['k'], atom_mapping[k]) & \
               isin(dihedraltypes['l'], atom_mapping[l]) & \
               isin(dihedraltypes['func'], func))
         if verbose:
            if any(mask):
               print(f'found complete match for {(i,j,k,l,func)}')
               print(f'{" ".join([atom_mapping[atom] for atom in [i, j, k, l]])}')
               print(dihedraltypes[mask])
            elif any(mask_Xl):
               print(f'found match three for {(i, j, k, l,func)}')
               print(f'{" ".join([atom_mapping[atom] for atom in [i, j, k, l]])}')
               print(dihedraltypes[mask_Xl])
            elif any(mask_Xll):
               print(f'found match two for {(i, j, k, l,func)}')
               print(f'{" ".join([atom_mapping[atom] for atom in [i, j, k, l]])}')
               print(dihedraltypes[mask_Xll])
            else:
               raise LookupError(f'Dihedral not found: {[i, j, k, l, func].__str__()} {[atom_mapping[atom] for atom in [i, j, k, l]].__str__()}')
         elif not any(mask | mask_Xl | mask_Xll):
            raise LookupError(f'Dihedral not found: {[i, j, k, l, func].__str__()} {[atom_mapping[atom] for atom in [i, j, k, l]].__str__()}')
         
         if any(mask):
            for entries in dihedraltypes[mask]:
               data.append((i, j, k, l, func, entries[5], entries[6], entries[7])) 
         elif any(mask_Xl):
            for entries in dihedraltypes[mask_Xl]:
               data.append((i, j, k, l, func, entries[5], entries[6], entries[7])) 
         else:
            for entries in dihedraltypes[mask_Xll]:
               data.append((i, j, k, l, func, entries[5], entries[6], entries[7]))
            
      if func == 9:
         mask = ((isin(dihedraltypes['i'], atom_mapping[i]) & \
               isin(dihedraltypes['j'], atom_mapping[j]) & \
               isin(dihedraltypes['k'], atom_mapping[k]) & \
               isin(dihedraltypes['l'], atom_mapping[l]) & \
               isin(dihedraltypes['func'], func)) | \
               (isin(dihedraltypes['i'], atom_mapping[l]) & \
               isin(dihedraltypes['j'], atom_mapping[k]) & \
               isin(dihedraltypes['k'], atom_mapping[j]) & \
               isin(dihedraltypes['l'], atom_mapping[i]) & \
               isin(dihedraltypes['func'], func))) & isin(dihedraltypes['func'], func)
         mask_Xlr = ((isin(dihedraltypes['i'], 'X') & \
               isin(dihedraltypes['j'], atom_mapping[j]) & \
               isin(dihedraltypes['k'], atom_mapping[k]) & \
               isin(dihedraltypes['l'], 'X') & \
               isin(dihedraltypes['func'], func)) | \
               (isin(dihedraltypes['i'], 'X') & \
               isin(dihedraltypes['j'], atom_mapping[k]) & \
               isin(dihedraltypes['k'], atom_mapping[j]) & \
               isin(dihedraltypes['l'], 'X') & \
               isin(dihedraltypes['func'], func))) & isin(dihedraltypes['func'], func)
         if verbose:
            if any(mask):
               print(f'found complete match for {(i,j,k,l,func)}')
               print(f'{" ".join([atom_mapping[atom] for atom in [i, j, k, l]])}')
               print(dihedraltypes[mask])
            elif any(mask_Xlr):
               print(f'found match two for {[i, j, k, l]}')
               print(f'{" ".join([atom_mapping[i] for i in [i, j, k, l]])}')
               print(dihedraltypes[mask_Xlr])
            else:
               raise LookupError(f'Dihedral not found: {[i, j, k, l, func].__str__()} {[atom_mapping[atom] for atom in [i, j, k, l]].__str__()}')
         elif not any(mask | mask_Xlr):
            raise LookupError(f'Dihedral not found: {[i, j, k, l, func].__str__()} {[atom_mapping[atom] for atom in [i, j, k, l]].__str__()}')
         if any(mask): 
            for entries in dihedraltypes[mask]:
               data.append((i, j, k, l, func, entries[5], entries[6], entries[7])) 
         else:
            for entries in dihedraltypes[ mask_Xlr]:
               data.append((i, j, k, l, func, entries[5], entries[6], entries[7])) 
      
   
   structured_array = array(data, dtype=dtype_dihedrals)
   structured_array['func'] *= -1
   structured_array.sort(order=['l','k','j','mult','i'])
   structured_array.sort(order='func')
   structured_array['func'] *= -1
   
   return structured_array

def dihedraltypes_strings_list(dihedraltypes):
   stringsout = [f'{dihedraltype[0]:<5} {dihedraltype[1]:<5} {"s"+dihedraltype[2]:<5} {"s"+dihedraltype[3]:<5} {dihedraltype[4]:^9}' + \
                 f'{dihedraltype[5]:<10}{dihedraltype[5]:<10.5f}{dihedraltype[7]}\n' \
                 for dihedraltype in dihedraltypes]
   return stringsout

def dihedrals_strings_list(dihedrals):
   stringsout = [f'{dihedral[0]:<5} {dihedral[1]:<5} {dihedral[2]:<5} {dihedral[3]:<5} {dihedral[4]:^9}' + \
                 f'{dihedral[5]:<10.3f}{dihedral[6]:<10.5f}{dihedral[7]}\n' \
                 if len(dihedral) == 8 else f'{dihedral[0]:<5} {dihedral[1]:<5} {dihedral[2]:<5} {dihedral[3]:<5} {dihedral[4]:^9}' \
                 for dihedral in dihedrals ]
   return stringsout
 
def topology_writer(**kwargs):
   ofile = kwargs.get('outfile', 'topol')
   filepath = kwargs.get('filepath', './')
   lambda_i = kwargs.get('lambda_i', None)
   kappa_i = kwargs.get('kappa_i', None)
   lines = kwargs.get('lines', None)
   verbose = kwargs.get('verbose', False)
   if any(lines):
      fullpath = f'{filepath}{ofile}'
      for additional in [lambda_i, kappa_i]: fullpath = f'{fullpath}-{additional:0.3f}' if additional else fullpath
      fullpath += ".top"
      with open(fullpath,'w') as file:
         file.writelines(lines)
      if verbose: print(f'Saved {fullpath}')
   else: print('try again with self.run()')
   

class parse():
   def __init__(self, ifile:str='processed.top', **kwargs):
      '''Parse GMX Topology'''

      self._processed = False
      self.nmol = None
      self.molecules = None
      self._sections = {}
      self._sections_out = {}
      
      self.hard_order_sections = [ "defaults", "atomtypes", "nonbond_params", "bondtypes", \
                                   "constrainttypes", "angletypes", "dihedraltypes","system", "molecules", "moleculetype"]
      with open(ifile) as topo:
         self.readfile = topo.readlines()
      self._gather_param_sections()
      self._atomtypes = [i[:i.find(';')].split() \
                               for i in self._sections['atomtypes'] if ';' not in i[:3] if '[' not in i[:3] \
                               if '\n' not in i[:3]]
      self._ordered_molecules()
      self.molecule_atoms = True
      if kwargs.get('verbose', False):
         print('topology file read')
      
   @property
   def molecule_atoms(self):
      return self._molecule_atoms
   
   @molecule_atoms.setter
   def molecule_atoms(self, a):
      if a == True:
         atdict = {}
         for nmol in range(len(self._sections['moleculetype'])):
            atdict[nmol] = {int(i.split()[0]): i.split()[1] for i in self._sections['moleculetype'][nmol]['atoms'] if len(i.split()) != 0 and ';' not in i[:5]}
         self._molecule_atoms = atdict
      pass

   def _populate_out(self, **kwargs):
      if kwargs.get('verbose', False):
         method = kwargs.get('method', None)
         if not method == 'solvent_scaled':
            print(f'Temperature Ladder:         {" ".join(self.templadder)}')
            print(f'Corresonding Lambdas:       {" ".join(self.lambdai)}')
         if kwargs.get('method',None) in [i for i in self._methods.keys() if i != 'rest2']:
            print(f'Corresponding kappa values: {" ".join(self.kappa)}')
         print(f'Produce {self.nreps} replicas')
      for lambda_i in self.lambdai:
         lines=self._sections['defaults']+['\n']
         lines+=['[ atomtypes ]\n']+self._scaled_atomtypes[lambda_i]+['\n']
         lines+=['[ nonbond_params ]\n']+self._scaled_nonbonded[lambda_i]+['\n']
         lines+=self._sections['bondtypes']+['\n']
         lines+=self._sections['constrainttypes']+['\n']
         lines+=self._sections['angletypes']+['\n']
         lines+=self._sections['dihedraltypes']+['\n']
         for molecule in self._scaled_molecules[lambda_i].keys():
            for section in self._scaled_molecules[lambda_i][molecule].keys():
               if section=='header':
                  lines+=self._scaled_molecules[lambda_i][molecule][section]+['\n']
               else:
                  lines+=[f"[ {section} ]\n"]+self._scaled_molecules[lambda_i][molecule][section]+['\n']
         lines+=self._sections['system']+['\n']
         lines+=self._sections['molecules']+['\n']
         self._sections_out[lambda_i]=lines
         
   def _gather_atomtypes_lines(self, atomtypes):
      atomtypes_lines = {}
      for line in self._atomtypes:
         if any(item in line for item in atomtypes):
            atomtypes_lines[line[0]] = line
      return atomtypes_lines
   
   def show_molecule_atomtypes(self, molecule:int=0):
      for i in enumerate(get_molecule_atomtypes(self._sections, molecule)): print('{}: {}'.format(*i))
      pass
   
   def _ordered_molecules(self):
      try:
         molecules = [" ".join(i.split()[:-1]) for i in self._sections['molecules'] if '[' not in i and ';' not in i.split()[0]] 
         self.molecules = {num: i for num,i in enumerate(molecules)}
         self.nmol = max(self.molecules.keys())
      except:
         print("Do you have molecules?")
      pass
   
   def show_molecule_names(self):
      try:
         print("\n".join([f'{key}: {mol}' for key,mol in zip(self.molecules.keys(),self.molecules.values())]))
      except:
         print("Do you have molecules?")
      pass
   
   def _moleculetype_sub(self,linestart:int):
      first_round = True
      output = []
      for line in self.readfile[linestart:]:
         if "[" not in line and ';' not in line[:3]:
            output.append(line)
         elif ";" == line[0]:
            continue
         elif "[" in line and first_round!=True: 
            break
         first_round = False
      return output 
   
   def _identify_moltype_sections(self,trunks:int):
      section_start = []
      for i, line in enumerate(self.readfile[trunks:]):
         if '[' in line and 'moleculetype' not in line and 'system' not in line:
            section_start.append(i+trunks)
         elif i != 0 and 'moleculetype' in line or 'system' in line:
            break
      return section_start

   def _parse_moleculetypes(self,trunks:int):
      output = {}
      sections = self._identify_moltype_sections(trunks)
      output['header'] = self.readfile[trunks:trunks+3]
      for section in sections:
         output[self.readfile[section].split()[1]] = []
      for section in sections:
         section_ = self.readfile[section].split()[1]
         output[section_] += self._moleculetype_sub(section)
      return output

   def _gather_param_sections(self):
      for section in self.hard_order_sections[:-1]:
         is_select = [ i for i, line in enumerate(self.readfile) if section in line ]
         in_select = [f' [ {section} ] \n']
         for i in is_select:
            in_select += _parse_section(self.readfile,i)
         self._sections[section]=in_select
      section = self.hard_order_sections[-1]
      is_select = [ i for i, line in enumerate(self.readfile) if section in line ] 
      moltype_dict = {} 
      for i in range(len(is_select)):
         moltype_dict[i] = self._parse_moleculetypes(is_select[i])
      self._sections[section] = moltype_dict
      pass
   

In [22]:
topology = parse()

In [23]:
lj_gmxtolmps = lambda x: array(topology._gather_atomtypes_lines([x]).popitem()[1][-2:], dtype=float)*array([10.0, 1/4.184])

In [24]:
topology._gather_atomtypes_lines(topology.molecule_atoms[0].values())

{'C': ['C', '6', '12.01', '0.0000', 'A', '3.39967e-01', '3.59824e-01'],
 'C6': ['C6', '6', '12.01', '0.0000', 'A', '3.39967e-01', '3.59824e-01'],
 'C5': ['C5', '6', '12.01', '0.0000', 'A', '3.39967e-01', '3.59824e-01'],
 'CA': ['CA', '6', '12.01', '0.0000', 'A', '3.39967e-01', '3.59824e-01'],
 'CT': ['CT', '6', '12.01', '0.0000', 'A', '3.39967e-01', '4.57730e-01'],
 'C1': ['C1', '6', '12.01', '0.0000', 'A', '3.39967e-01', '4.57730e-01'],
 'C7': ['C7', '6', '12.01', '0.0000', 'A', '3.39967e-01', '4.57730e-01'],
 'C8': ['C8', '6', '12.01', '0.0000', 'A', '3.39967e-01', '4.57730e-01'],
 'C9': ['C9', '6', '12.01', '0.0000', 'A', '3.39967e-01', '4.57730e-01'],
 'H': ['H', '1', '1.008', '0.0000', 'A', '1.06908e-01', '6.56888e-02'],
 'HB': ['HB', '1', '1.008', '0.0000', 'A', '1.06908e-01', '6.56888e-02'],
 'HC': ['HC', '1', '1.008', '0.0000', 'A', '2.64953e-01', '6.56888e-02'],
 'H1': ['H1', '1', '1.008', '0.0000', 'A', '2.47135e-01', '6.56888e-02'],
 'HA': ['HA', '1', '1.008', '0.0000', 'A',

In [25]:
lj_lmps_values = list(map(lj_gmxtolmps, topology.molecule_atoms[0].values()))

In [26]:
protein = mda.Universe('apo-slab-protein.renum.gro')

In [27]:
xyz = protein.atoms.positions

In [28]:
xyz

array([[  6.7000003,  40.25     , 111.91     ],
       [  7.13     ,  39.83     , 112.729996 ],
       [  5.93     ,  39.66     , 111.630005 ],
       ...,
       [148.59999  , 101.51     , 107.89     ],
       [147.67     , 101.23     , 107.1      ],
       [149.59     , 102.21     , 107.58     ]], dtype=float32)

In [43]:
gfg = concatenate([xyz, concatenate([lj_lmps_values]*40)], axis=1)